In [ ]:
import os
current_dir = os.getcwd()
current_dir

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

In [ ]:
current_dir = os.getcwd()
current_dir

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.read_csv('/Users/mohammedluqmanriaz/Desktop/Code Instatute/capstone_try_two/2025_capstone_project/jupyter_notebooks/Data_set/transactions.csv')  # Make sure this path is correct


In [ ]:
# Function to create a data dictionary
def create_data_dictionary(df):
    # Optional: Provide known descriptions
    descriptions = {
        'InvoiceNo': 'Invoice number - 6-digit integral number uniquely assigned to each transaction. If starts with letter "c", indicates a cancellation',
        'StockCode': 'Product (item) code - 5-digit integral number uniquely assigned to each distinct product',
        'Description': 'Product (item) name/description',
        'Quantity': 'The quantities of each product (item) per transaction',
        'InvoiceDate': 'Invoice date and time - the day and time when each transaction was generated',
        'UnitPrice': 'Unit price - product price per unit in sterling (£)',
        'CustomerID': '5-digit integral number uniquely assigned to each customer',
        'Country': 'Country name - the name of the country where each customer resides'
    }

    dictionary_data = []
    for column in df.columns:
        sample_values = df[column].dropna().head(3).tolist()
        sample_str = ', '.join([str(x) for x in sample_values])
        
        dictionary_data.append({
            'Column': column,
            'Data Type': str(df[column].dtype),
            'Missing Values': df[column].isnull().sum(),
            'Missing %': round((df[column].isnull().sum() / len(df)) * 100, 2),
            'Unique Values': df[column].nunique(),
            'Sample Values': sample_str,
            'Description': descriptions.get(column, 'Custom/Engineered column - description needed')
        })
        
    return pd.DataFrame(dictionary_data)

# Generate the data dictionary
raw_data_dictionary = create_data_dictionary(df)

# Display or export
print(raw_data_dictionary) # This will be removed in the final version
# raw_data_dictionary.to_csv('data_dictionary.csv', index=False) # Uncomment to save to CSV

In [ ]:
# Check for duplicate customer IDs
duplicate_customer_ids = df[df.duplicated(subset='customer_id', keep=False)]
duplicate_customer_count = duplicate_customer_ids['customer_id'].nunique()
print(f"\nNumber of duplicate customer IDs: {duplicate_customer_count}")
print("Duplicate customer ID records:")
print(duplicate_customer_ids.sort_values(by='customer_id').head(10))  # Display first 10 for brevity

# Check for duplicate transaction IDs
duplicate_transaction_ids = df[df.duplicated(subset='transaction_id', keep=False)]
duplicate_transaction_count = duplicate_transaction_ids['transaction_id'].nunique()
print(f"\nNumber of duplicate transaction IDs: {duplicate_transaction_count}")
if duplicate_transaction_count > 0:
    print("Duplicate transaction ID records:")
    print(duplicate_transaction_ids.sort_values(by='transaction_id'))
else:
    print("No duplicate transaction IDs found.")